# CC6204 - 2018 Universidad de Chile
## Código de Red Neuronal simple estilo pytorch
El siguiente código se entrega a modo de ejemplo para crear una red neuronal Feed Forward de 2 capas escondidas y predicción binaria. Todos los parámetros de la red se crean a mano, pero dejamos que pytorch calcule los gradientes que luego usamos en el loop principal de entrenamiento. Incluye el soporte para GPUs.

La idea es que el código sirva para aprender la arquitectura general de red+entrenamiento+predicción usando las clases abstractas de pytorch además de las utilidades que entrega para crear optimizadores y objetos para cargar datos de manera eficiente. 

El código muestra varias cosas:

*   uso de funciones de activación y pérdida en `torch.nn` (
[`torch.nn.Sigmoid`](http://pytorch.org/docs/master/nn.html#torch.nn.Sigmoid),
[`torch.nn.Tanh`](http://pytorch.org/docs/master/nn.html#torch.nn.Tanh) y
[`torch.nn.BCELoss`](http://pytorch.org/docs/master/nn.html#torch.nn.BCELoss)),
*   el estilo pytorch de definir redes (heredando de [`torch.nn.Module`](http://pytorch.org/docs/master/nn.html#torch.nn.Module)),
*   el estilo pytorch de definir optimizadores (heredando de [`torch.optim.Optimizer`](http://pytorch.org/docs/master/optim.html)),
*   la creación de datasets (heredando de [`torch.utils.data.Dataset`](http://pytorch.org/docs/master/data.html)) y el uso de dataloaders ([`torch.utils.data.DataLoader`](http://pytorch.org/docs/master/data.html)).

(Pensado para correr en [Colaboratory](http://colab.research.google.com))



In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install -q ipdb

In [0]:
import torch
import numpy as np
import sys
import time
import ipdb

# Estos dos imports son para clases útiles.
from torch.nn import Parameter
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

# Genera una semilla fija para que los experimentos sea repetibles.
t_cg = torch.manual_seed(1547)

In [3]:
# Chequeamos si hay acceso a la GPU
print(torch.cuda.is_available())

True


In [0]:
# torch.nn tiene varias funciones de activación y de pérdida
# que se pueden utilizar directamente. No es necesario programar
# las más comunes.

# Funciones de activación
sig = torch.nn.Sigmoid()
tanh = torch.nn.Tanh()

# Función de pérdida (Binary Cross Entropy Loss)
cross_ent = torch.nn.BCELoss()

In [0]:
# Las redes (modelos en general) deben heredar desde torch.nn.Module
class FFNN2C(torch.nn.Module):
  
  # En el inicializador debemos crear los parámetros de la red
  # así como todo lo necesario para hacer las predicciones.
  def __init__(self, d0, d1, d2):
    super(FFNN2C, self).__init__()
    
    # Un 'Parameter' es un caso particular de 'Variable' y hace que los
    # parámetros se agreguen automáticamente al iterador .parameters()
    # de la red.
    self.W1 = Parameter(torch.randn(d0,d1))
    self.c1 = Parameter(torch.zeros(d1))
    self.W2 = Parameter(torch.randn(d1,d2))
    self.c2 = Parameter(torch.zeros(d2))
    self.W3 = Parameter(torch.randn(d2,1))
    
  def forward(self, x):
    u1 = x.mm(self.W1).add(self.c1)
    h1 = tanh(u1)
    u2 = h1.mm(self.W2).add(self.c2)
    h2 = sig(u2)
    u3 = h2.mm(self.W3)
    y_pred = sig(u3)
    
    return y_pred
  
# Las deribadas necesarias para el Back Propagation se calculan 
# de manera automática con Module.backward()!!!! :-)

In [0]:
# Los optimizadores deben heredar desde torch.optim.Optimizer

# Acá definimos uno muy simple de descenso de gradiente simple.
class DG(torch.optim.Optimizer):
  def __init__(self, params, lr):
    # Con esta llamada los parámetros quedan en 
    # self.param_groups[i]['params']
    super(DG, self).__init__(params, {'lr':lr})
  
  def step(self):
    # Este optimizador usa un solo conjunto de paramteros.
    # Para un optimizador con más grupos de parámetros se puede
    # iterar sobre self.param_groups
    params = self.param_groups[0]['params']
    lr = self.param_groups[0]['lr']
    for p in params:
        if p.grad is None:
          continue
        p.data -= p.grad.data * lr
        
# pytorch tiene varios otros optimizadores ya programados que se
# pueden utilizar (incluído el torch.optim.SGD similar al DGB que 
# acabamos de programar).

In [0]:
# Finalmente para cargar los datos, pytorch provee varias clases
# que simplifican el proceso. Las mas importantes son Dataset (clase
# abstracta) y DataLoader. 
# A continuación creamos un Dataset aleatorio.
class RandomDataSet(Dataset):
  def __init__(self, N, f):
    R_N_f = torch.rand(N,f)
    self.X = torch.bernoulli(R_N_f)
    R_N_1 = torch.rand(N,1)
    self.Y = torch.bernoulli(R_N_1)
    
  # Debemos definir __len__ para retornar la cantidad de datos/ejemplos
  # en nuestro dataset.
  def __len__(self):
    return self.X.size()[0]

  # Debemos definir __getitem__ para retornar el i-ésimo ejemplo en nuestro
  # dataset. En este caso los datos los sacamos de la memoria principal,
  # pero podríamos sacarlos desde la web, el disco, o incluso irlos generando
  # a medida que los van solicitando.
  def __getitem__(self, i):
    return self.X[i], self.Y[i]

In [0]:
# Ya tenemos todo listo para definir nuestra iteración principal de
# entrenamiento de la red.
def ejemplo_FFNN(dataset, features, d1=200, d2=300, lr=0.06, b=1,
                 epochs=10, run_in_GPU=False, report_every=1,
                 verbose=False, cheq_grad=False,
                 debug=False):
  
  # Crea la red
  red = FFNN2C(features,d1,d2)
  # Como heredamos desde torch.nn.Module, pasar todos los parámetros de la
  # red a la GPU es muy simple, simplemente usando .cuda().
  if run_in_GPU:
    red = red.cuda()
  
  # Crea el optimizador de parametros para la red.
  optimizador = DG(red.parameters(), lr)
  
  # Usamos DataLoader para crear un iterador sobre el dataset. En este
  # caso usamos un batch de tamaño 'b' y le pedimos que desordene los datos.
  batches = DataLoader(dataset, batch_size=b, shuffle=True)  

  # Comienzo del entrenamiento.
  tiempo_epochs = 0
  for e in range(1,epochs+1):
    inicio_epoch = time.clock()
    
    for x, y in batches:
      x = Variable(x, requires_grad=False)
      y = Variable(y, requires_grad=False)
      
      # Pásalos a la GPU si fuera necesario.
      if run_in_GPU:
        x = x.cuda()
        y = y.cuda()

      # Usa la red para computar la predicción.
      y_pred = red(x)

      # Calcula la función de pérdida.
      L = cross_ent(y_pred, y)
        
      # Vacía los gradientes.
      red.zero_grad()
      # Vaciar los gradientes es muy importante pues pytorch nos permite
      # tener control total sobre los gradientes que vamos computando y
      # por ejemplo acumularlos desde distintas redes.

      # Computa la pasada hacia atrás.
      L.backward()

      # Computa un paso del optimizador (modifica los pesos).
      optimizador.step()

      # Listo! :-)
    tiempo_epochs += time.clock() - inicio_epoch
    
    if e % report_every == 0:
      # Podemos ir chequeando la certeza de las predicciones hasta el momento.
      # En este caso las chequeamos sobre todos los ejemplos.
      # Ojo: no es lo habitual hacerlo así pues se nos puede llenar la memoria
      # si cargamos todos los datos juntos. Esto es solo demostrativo.
      X = data.X
      Y = data.Y
      
      # Pásalo a la GPU si fuera necesario
      if run_in_GPU:
        X = X.cuda()
        Y = Y.cuda()

      Xv = Variable(X)
      Yv = Variable(Y)    
      Y_PREDv = red(Xv)

      L = cross_ent(Y_PREDv, Yv).data[0]

      # Convierte las predicciones en 0 o 1 dependiendo de si la probabilidad
      # es mayor o igual a 0.5.
      Y_PRED = (Y_PREDv.data >= 0.5).float()    
      correctos = torch.sum(Y_PRED == Y)
      acc = float(correctos) / float(N) * 100

      sys.stdout.write(
          '\rEpoch:{0:03d}'.format(e) + ' Acc:{0:.2f}%'.format(acc)
          + ' Loss:{0:.4f}'.format(L) 
          + ' Tiempo/epoch:{0:.3f}s'.format(tiempo_epochs/e))
    

In [10]:
# Ahora la probamos.
N = 5000 
f = 300
data = RandomDataSet(N,f)
ejemplo_FFNN(data,f,b=32,d1=300,d2=400,epochs=30,run_in_GPU=False)

Epoch:030 Acc:99.98% Loss:0.0164 Tiempo/epoch:0.332s